In [ ]:
# Import necessary modules
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read in the image
img = cv2.imread('data/background.png')
img1 = cv2.imread('images/o_74.png')
img2 = cv2.imread('images/m_74.png')

In [ ]:
#測試
img1= cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
# img2= cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
img1 = cv2.GaussianBlur(img1, (3, 3), 0)
img2 = cv2.GaussianBlur(img2, (7, 7), 0)
img1= cv2.Canny(img1, 300,150)
# img2= cv2.Canny(img2, 300,150)


In [ ]:
#測試
plt.imshow(img1, cmap = 'gray')
plt.show()

In [ ]:
#測試
plt.imshow(img2, cmap = 'gray')
plt.show()

In [ ]:
#線段數據
lane = {
    "lane1":{
        "top":{
            "x":529,
            "y":231
        },
        "bot":{
            "x":430,
            "y":534
        },
        "slope":"x=(183156-99*y)/303"
    },
    "lane2":{
        "top":{
            "x":559,
            "y":231
        },
        "bot":{
            "x":628,
            "y":534
        },
        "slope":"x=(153438+69*y)/303"
    },
    "lane3":{
        "top":{
            "x":589,
            "y":231
        },
        "bot":{
            "x":830,
            "y":534
        },
        "slope":"x=(138816+211*y)/303"
    },
    "lane4":{
        "top":{
            "x":619,
            "y":231
        },
        "bot":{
            "x":958,
            "y":465
        },
        "slope":"x=(65918+339*y)/233"
    },
}

In [ ]:
#計算車輛位置
def CountObject(lane):
    index = []
    have_object = False
    count = 0
    object_count = 0
    for i in range(lane.size):
        if not have_object:
            if lane[i] < 100:
                continue
            else:
                have_object = True
        else:
            if lane[i] < 40:
                index.append(i)
                have_object = False
    return index


In [ ]:
#車距違規判斷
def validation(index):
    if len(index)==0 or len(index)==1:
        return "pass"
    baseline = [260, 269, 282, 297, 318, 346, 395, 462]
    count = -1
    now_ind = False
    danger = False
    #所有index尋找自己在哪兩個baseline區間
    for i in range(len(index)):
        #尋找baseline區間
        for j in range(1, len(baseline)):
            if baseline[j-1] < index[i] <= baseline[j]:
                #若index[i]在區間內，則剩餘index尋找是否有在往後推的三格區間內
                for k in range(i+1, len(index)):
                    for l in range(3):
                        #若無法在往後尋找則continue
                        if j+l >= len(baseline):
                            continue
                        #若真的在三格區間內找到其他index則代表danger
                        if baseline[j-1+l] < index[k] <= baseline[j+l]:
                            return 'danger'
    return 'pass'

In [ ]:
#主要邏輯
def DetectLane(image):
    lane1 = np.zeros(230)
    lane2 = np.zeros(230)
    lane3 = np.zeros(230)
    lane4 = np.zeros(230)
    for i in range(231, 535):
        lane1 = np.append(lane1, image[i][(int)((183156-99*i)/303)])
        lane2 = np.append(lane2, image[i][(int)((153438+69*i)/303)])
        lane3 = np.append(lane3, image[i][(int)((138816+211*i)/303)])
    for i in range(231, 465):
        lane4 = np.append(lane4, image[i][(int)((65918+339*i)/233)])
    # plt.subplot(7, 1, 1)
    # plt.plot(lane1)
    # plt.title('Lane1')
    # plt.subplot(7, 1, 3)
    # plt.plot(lane2)
    # plt.title('Lane2')
    # plt.subplot(7, 1, 5)
    # plt.plot(lane3)
    # plt.title('Lane3')
    # plt.subplot(7, 1, 7)
    # plt.plot(lane4)
    # plt.title('Lane4')
    # plt.show()
    ind1 = CountObject(lane1)
    ind2 = CountObject(lane2)
    ind3 = CountObject(lane3)
    ind4 = CountObject(lane4)
    # print(ind1, ind2, ind3, ind4)
    return [validation(ind1),validation(ind2),validation(ind3),validation(ind4)]

In [ ]:
#計算標線位置
def GenScole():
    y = 282
    x1=(int)((183156-99*y)/303)
    x2=(int)((153438+69*y)/303)
    x3=(int)((138816+211*y)/303)
    x4=(int)((65918+339*y)/233)
    print(x1, x2, x3, x4)

In [ ]:
#繪製標線
def DrawLine():
    plt.plot([430, 529], [534, 231], color="red")
    plt.plot([628, 559], [534, 231], color="red")
    plt.plot([830, 589], [534, 231], color="red")
    plt.plot([958, 619], [465, 231], color="red")
    plt.plot([374, 957], [462, 462], color="yellow", linestyle="--")
    plt.plot([418, 899], [395, 395], color="yellow", linestyle="--")
    plt.plot([446, 820], [346, 346], color="yellow", linestyle="--")
    plt.plot([466, 781], [318, 318], color="yellow", linestyle="--")
    plt.plot([476, 730], [297, 297], color="yellow", linestyle="--")
    plt.plot([486, 710], [282, 282], color="yellow", linestyle="--")
    plt.plot([495, 695], [269, 269], color="yellow", linestyle="--")
    plt.plot([499, 680], [260, 260], color="yellow", linestyle="--")

In [ ]:
#測試 #debbuging with img2
def Test(img2):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 
    DetectLane(img2)
    plt.imshow(img2, cmap = 'gray')
    DrawLine()
    plt.show()
Test(img2)

In [ ]:
#回傳顏色
def state_color(state):
    return (0, 255, 0) if state == 'pass' else (0, 0, 255)

In [ ]:
#Main
import os
fourcc1 = cv2.VideoWriter_fourcc(*'MJPG')
out1 = cv2.VideoWriter('./videos/output1.avi', fourcc1, 20.0, (960,540))
while(1):
    exit = False
    for i in range (0, 134):
        if os.path.isfile(f'./images/m_{i}.png'):
            image_o = cv2.imread(f'./images/o_{i}.png')
            image_m = cv2.imread(f'./images/m_{i}.png')
            image_m = cv2.cvtColor(image_m, cv2.COLOR_BGR2GRAY)
            image_m = cv2.GaussianBlur(image_m, (7, 7), 0)
            state = DetectLane(image_m)
            cv2.circle(image_o, (435, 485), 20, state_color(state[0]), -1)
            cv2.circle(image_o, (615, 485), 20, state_color(state[1]), -1)
            cv2.circle(image_o, (790, 485), 20, state_color(state[2]), -1)
            cv2.circle(image_o, (905, 440), 20, state_color(state[3]), -1)
            out1.write(image_o)
            cv2.imshow('Predict', image_o)
            if cv2.waitKey(1) & 0xFF == 27:
                exit = True
                break
            
        if i == 133:
            exit = True
            break
    if exit:
        break
out1.release()
cv2.destroyAllWindows()